In [183]:
from bs4 import BeautifulSoup
import requests

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

def url_rent(page):
    return 'https://www.zillow.com/homes/for_rent/Oakland-CA/all_type/13072_rid/37.971537,-121.981545,37.612327,-122.47593_rect/10_zm/' + page + '_p/'


In [186]:
all_prices = []
all_addresses = []

for page in np.arange(1, 21):
    with requests.Session() as s:
        url = url_rent(str(page))
        r = s.get(url, headers=req_headers)
        
        soup = BeautifulSoup(r.content, 'lxml')
        price = soup.find_all('span', {'class': 'zsg-photo-card-price'})
        address = soup.find_all('span', {'itemprop': 'address'})
        
        for i in np.arange(len(price)):
            home_price = price[i].text
            home_address = address[i].text
            
            all_prices.append(home_price)
            all_addresses.append(home_address)

In [187]:
soup

<html><head><meta content="noindex, nofollow" name="robots"/><script src="https://www.google.com/recaptcha/api.js"></script><link href="https://www.zillowstatic.com/vstatic/18f7df5/static/css/z-pages/captcha.css" media="screen" rel="stylesheet" type="text/css"/><script>
            function handleCaptcha(response) {
                var vid = getQueryString("vid"); // getQueryString is implemented below
                var uuid = getQueryString("uuid");
                var name = '_pxCaptcha';
                var cookieValue =  btoa(JSON.stringify({r:response,v:vid,u:uuid}));
                var cookieParts = [name, '=', cookieValue, '; path=/'];
                cookieParts.push('; domain=' + window.location.hostname);
                cookieParts.push('; max-age=10');//expire after 10 seconds
                document.cookie = cookieParts.join('');
                var originalURL = getOriginalUrl("url");
                var originalHost = window.location.host;
                var newHref

In [189]:
from bs4 import BeautifulSoup
import requests

req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as s:
    url = 'https://www.zillow.com/homes/for_sale/Los-Angeles-CA_rb/?fromHomePage=true&shouldFireSellPageImplicitClaimGA=false&fromHomePageTab=buy'
    r = s.get(url, headers=req_headers)

soup = BeautifulSoup(r.content, 'lxml')
# price = soup.find('span', {'class': 'zsg-photo-card-price'}).text
# info = soup.find('span', {'class': 'zsg-photo-card-info'}).text
# address = soup.find('span', {'itemprop': 'address'}).text

In [190]:
soup

<html><head><meta content="noindex, nofollow" name="robots"/><script src="https://www.google.com/recaptcha/api.js"></script><link href="https://www.zillowstatic.com/vstatic/18f7df5/static/css/z-pages/captcha.css" media="screen" rel="stylesheet" type="text/css"/><script>
            function handleCaptcha(response) {
                var vid = getQueryString("vid"); // getQueryString is implemented below
                var uuid = getQueryString("uuid");
                var name = '_pxCaptcha';
                var cookieValue =  btoa(JSON.stringify({r:response,v:vid,u:uuid}));
                var cookieParts = [name, '=', cookieValue, '; path=/'];
                cookieParts.push('; domain=' + window.location.hostname);
                cookieParts.push('; max-age=10');//expire after 10 seconds
                document.cookie = cookieParts.join('');
                var originalURL = getOriginalUrl("url");
                var originalHost = window.location.host;
                var newHref

In [129]:
url = url_rent('10')

In [130]:
url

'https://www.zillow.com/homes/for_rent/37.911023,-122.124195,37.731489,-122.371387_rect/11_zm/10_p/'

In [131]:
'https://www.zillow.com/homes/for_rent/Oakland-CA/all_type/13072_rid/37.971537,-121.981545,37.612327,-122.47593_rect/10_zm/1_p/'

'https://www.zillow.com/homes/for_rent/Oakland-CA/all_type/13072_rid/37.971537,-121.981545,37.612327,-122.47593_rect/10_zm/1_p/'